In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
from dark_emulator_public import dark_emulator
import os, sys, time, json
import matplotlib.pyplot as plt
from collections import OrderedDict as od
from scipy.interpolate import InterpolatedUnivariateSpline as ius
from scipy.interpolate import interp2d, interp1d
from scipy.integrate import simps
from tqdm import tqdm
import hsc3x2pt

using dark_emulator at  /Users/sunao-mac/Documents/python_package/dark_emulator_public/dark_emulator/__init__.py


In [3]:
t0 = time.time()

## Preparation for Fisher
Computation of $C(l)$ takes ~20 sec for each cosmology.
So this notebook pre-compute $C(l)$ for cosmologies needed for Fisher analysis.

Model parameters of 3x2pt analysis with single(double) source redshift bin are
- comsology: 5
- galaxy bias: 1x3
- magnification bias: 1x3
- photo-z: 1x1(2)
- multiplicative bais: 1x1(2)

13(15) parameters in total.

$C(l)$s depend on cosmological parameters, galaxy bias and magnification bias, for which we need to compute $C(l)$.

**Note** : Fisher analysis is doable even if the computational time for one model reaches ~20 sec. However, in the real analysis, ~20 sec is too much to run sampling code. 
I just want to know how people in cosmic shear, like Hikage-san and Hamana-san, or people in 3x2pt, like DES or KiDS, implement code of $C(l)$.?

In [4]:
power_b1 = hsc3x2pt.power_b1_class()

initialize cosmo_class
Initialize pklin emulator
initialize propagator emulator
Initialize sigma_d emulator
initialize cross-correlation emulator
initialize auto-correlation emulator
Initialize sigmaM emulator
initialize xinl emulator


In [5]:
Omega_s_HSC = 140

### List of models to compute $C(l)$ to compute **single** source bin anlaysis
0. fiducial = Planck cosmology + fiducial galaxy bias, magnification bais + 0 photo-z and multiplicative bias
1. $\omega_b h^2$ +
2. $\omega_c h^2$ +
3. $\Omega_{\rm de}$ +
4. $n_{\rm s}$ +
5. $\ln 10^{10}A_{\rm s}$ +
6. $b_{\rm 1,1}$ + 
7. $b_{\rm 1,2}$ + 
8. $b_{\rm 1,3}$ + 
9. $\alpha_{\rm mag,1}$ + 
10. $\alpha_{\rm mag,2}$ + 
11. $\alpha_{\rm mag,3}$ + 
12. $\Delta z_{\rm ph}$ +
13. $\Delta m$ + = easy. Just multiply $1+\Delta m$ to fiducial.

In [6]:
dirname = 'single_source_ClY1'

In [7]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
pk2cl.set_cosmology_from_dict()
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=True)
pk2cl.dump_Cl_cache(f'{dirname}/fiducial', overwrite=True)

:22.423463106155396 sec
saving l to single_source_ClY1/fiducial/l.txt
saving lowz,lowz to single_source_ClY1/fiducial/lowz,lowz.txt
saving lowz,cmass1 to single_source_ClY1/fiducial/lowz,cmass1.txt
saving lowz,cmass2 to single_source_ClY1/fiducial/lowz,cmass2.txt
saving cmass1,lowz to single_source_ClY1/fiducial/cmass1,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/fiducial/cmass1,cmass1.txt
saving cmass1,cmass2 to single_source_ClY1/fiducial/cmass1,cmass2.txt
saving cmass2,lowz to single_source_ClY1/fiducial/cmass2,lowz.txt
saving cmass2,cmass1 to single_source_ClY1/fiducial/cmass2,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/fiducial/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/fiducial/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/fiducial/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/fiducial/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/fiducial/s12,s12.txt
saving cosmo_dict to single_source_ClY1/fiducial/cosmo_dict.json.
saved galaxy

In [8]:
cosmo_fiducial = pk2cl.get_cosmo_dict()
dp = 0.01
file_param = od()

In [9]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['omega_b'] = cosmo_fiducial['omega_b']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['omega_b'] = cosmo_fiducial['omega_b']*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/omega_b', overwrite=True)

:13.1600980758667 sec
saving l to single_source_ClY1/omega_b/l.txt
saving lowz,lowz to single_source_ClY1/omega_b/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/omega_b/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/omega_b/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/omega_b/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/omega_b/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/omega_b/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/omega_b/s12,s12.txt
saving cosmo_dict to single_source_ClY1/omega_b/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.7 s, sys: 164 ms, total: 13.9 s
Wall time: 13.5 s


In [10]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['omega_c'] = cosmo_fiducial['omega_c']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['omega_c'] = cosmo_fiducial['omega_c']*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/omega_c', overwrite=True)

:13.073076963424683 sec
saving l to single_source_ClY1/omega_c/l.txt
saving lowz,lowz to single_source_ClY1/omega_c/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/omega_c/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/omega_c/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/omega_c/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/omega_c/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/omega_c/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/omega_c/s12,s12.txt
saving cosmo_dict to single_source_ClY1/omega_c/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.7 s, sys: 144 ms, total: 13.9 s
Wall time: 13.4 s


In [11]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['Omega_de'] = cosmo_fiducial['Omega_de']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['Omega_de'] = cosmo_fiducial['Omega_de']*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/Omega_de', overwrite=True)

:12.962203025817871 sec
saving l to single_source_ClY1/Omega_de/l.txt
saving lowz,lowz to single_source_ClY1/Omega_de/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/Omega_de/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/Omega_de/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/Omega_de/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/Omega_de/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/Omega_de/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/Omega_de/s12,s12.txt
saving cosmo_dict to single_source_ClY1/Omega_de/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.6 s, sys: 157 ms, total: 13.7 s
Wall time: 13.3 s


In [12]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['n_s'] = cosmo_fiducial['n_s']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['n_s'] = cosmo_fiducial['n_s']*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/n_s', overwrite=True)

:13.618571043014526 sec
saving l to single_source_ClY1/n_s/l.txt
saving lowz,lowz to single_source_ClY1/n_s/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/n_s/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/n_s/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/n_s/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/n_s/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/n_s/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/n_s/s12,s12.txt
saving cosmo_dict to single_source_ClY1/n_s/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 14.2 s, sys: 183 ms, total: 14.4 s
Wall time: 13.9 s


In [13]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['ln10p10As'] = cosmo_fiducial['ln10p10As']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['ln10p10As'] = cosmo_fiducial['ln10p10As']*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/ln10p10As', overwrite=True)

:12.484519958496094 sec
saving l to single_source_ClY1/ln10p10As/l.txt
saving lowz,lowz to single_source_ClY1/ln10p10As/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/ln10p10As/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/ln10p10As/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/ln10p10As/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/ln10p10As/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/ln10p10As/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/ln10p10As/s12,s12.txt
saving cosmo_dict to single_source_ClY1/ln10p10As/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.2 s, sys: 125 ms, total: 13.3 s
Wall time: 12.8 s


In [14]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78*(1.0+dp), '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['b1lowz'] = 1.78*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/b1lowz', overwrite=True)

:12.330716133117676 sec
saving l to single_source_ClY1/b1lowz/l.txt
saving lowz,lowz to single_source_ClY1/b1lowz/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/b1lowz/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/b1lowz/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/b1lowz/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/b1lowz/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/b1lowz/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/b1lowz/s12,s12.txt
saving cosmo_dict to single_source_ClY1/b1lowz/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13 s, sys: 117 ms, total: 13.2 s
Wall time: 12.6 s


In [15]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10*(1+dp), '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['b1cmass1'] = 2.12*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/b1cmass1', overwrite=True)

:12.676939010620117 sec
saving l to single_source_ClY1/b1cmass1/l.txt
saving lowz,lowz to single_source_ClY1/b1cmass1/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/b1cmass1/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/b1cmass1/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/b1cmass1/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/b1cmass1/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/b1cmass1/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/b1cmass1/s12,s12.txt
saving cosmo_dict to single_source_ClY1/b1cmass1/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.3 s, sys: 138 ms, total: 13.5 s
Wall time: 13 s


In [16]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28*(1+dp), '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['b1cmass2'] = 2.28*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/b1cmass2', overwrite=True)

:12.558178901672363 sec
saving l to single_source_ClY1/b1cmass2/l.txt
saving lowz,lowz to single_source_ClY1/b1cmass2/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/b1cmass2/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/b1cmass2/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/b1cmass2/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/b1cmass2/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/b1cmass2/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/b1cmass2/s12,s12.txt
saving cosmo_dict to single_source_ClY1/b1cmass2/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.2 s, sys: 133 ms, total: 13.4 s
Wall time: 12.9 s


In [17]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259*(1.0+dp)])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['alphamaglowz'] = 2.259*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/alphamaglowz', overwrite=True)

:12.42435622215271 sec
saving l to single_source_ClY1/alphamaglowz/l.txt
saving lowz,lowz to single_source_ClY1/alphamaglowz/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/alphamaglowz/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/alphamaglowz/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/alphamaglowz/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/alphamaglowz/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/alphamaglowz/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/alphamaglowz/s12,s12.txt
saving cosmo_dict to single_source_ClY1/alphamaglowz/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.1 s, sys: 121 ms, total: 13.3 s
Wall time: 12.7 s


In [18]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563*(1+dp)])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['alphamagcmass1'] = 3.563*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/alphamagcmass1', overwrite=True)

:12.731518983840942 sec
saving l to single_source_ClY1/alphamagcmass1/l.txt
saving lowz,lowz to single_source_ClY1/alphamagcmass1/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/alphamagcmass1/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/alphamagcmass1/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/alphamagcmass1/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/alphamagcmass1/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/alphamagcmass1/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/alphamagcmass1/s12,s12.txt
saving cosmo_dict to single_source_ClY1/alphamagcmass1/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.4 s, sys: 121 ms, total: 13.6 s
Wall time: 13 s


In [19]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729*(1+dp)])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['alphamagcmass2'] = 3.729*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/alphamagcmass2', overwrite=True)

[autoreload of hsc3x2pt failed: Traceback (most recent call last):
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<

:12.389916181564331 sec
saving l to single_source_ClY1/alphamagcmass2/l.txt
saving lowz,lowz to single_source_ClY1/alphamagcmass2/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/alphamagcmass2/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/alphamagcmass2/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/alphamagcmass2/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/alphamagcmass2/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/alphamagcmass2/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/alphamagcmass2/s12,s12.txt
saving cosmo_dict to single_source_ClY1/alphamagcmass2/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.1 s, sys: 108 ms, total: 13.2 s
Wall time: 12.7 s


In [20]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', dp, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['dzph'] = dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/dzph', overwrite=True)

[autoreload of hsc3x2pt failed: Traceback (most recent call last):
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<

:12.057708024978638 sec
saving l to single_source_ClY1/dzph/l.txt
saving lowz,lowz to single_source_ClY1/dzph/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/dzph/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/dzph/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/dzph/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/dzph/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/dzph/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/dzph/s12,s12.txt
saving cosmo_dict to single_source_ClY1/dzph/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 12.8 s, sys: 101 ms, total: 12.9 s
Wall time: 12.4 s


In [21]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, dp, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['dm'] = dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/dm', overwrite=True)

:12.349252223968506 sec
saving l to single_source_ClY1/dm/l.txt
saving lowz,lowz to single_source_ClY1/dm/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY1/dm/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY1/dm/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY1/dm/lowz,s12.txt
saving cmass1,s12 to single_source_ClY1/dm/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY1/dm/cmass2,s12.txt
saving s12,s12 to single_source_ClY1/dm/s12,s12.txt
saving cosmo_dict to single_source_ClY1/dm/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.1 s, sys: 105 ms, total: 13.2 s
Wall time: 12.6 s


In [22]:
json.dump(file_param, open(os.path.join(dirname, 'file_param.json'), 'w'), indent=2)

[autoreload of hsc3x2pt failed: Traceback (most recent call last):
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<

### List of models to compute $C(l)$ to compute **double** source bins anlaysis

0. fiducial = Planck cosmology + fiducial galaxy bias, magnification bais + 0 photo-z and multiplicative bias
1. $\omega_b h^2$ +
2. $\omega_c h^2$ +
3. $\Omega_{\rm de}$ +
4. $n_{\rm s}$ +
5. $\ln 10^{10}A_{\rm s}$ +
6. $b_{\rm 1,1}$ + 
7. $b_{\rm 1,2}$ + 
8. $b_{\rm 1,3}$ + 
9. $\alpha_{\rm mag,1}$ + 
10. $\alpha_{\rm mag,2}$ + 
11. $\alpha_{\rm mag,3}$ + 
12. $\Delta z_{\rm ph,1}$ +
12. $\Delta z_{\rm ph,2}$ +
13. $\Delta m_1$ + = easy. Just multiply $1+\Delta m_1$ to fiducial.
13. $\Delta m_2$ + = easy. Just multiply $1+\Delta m_2$ to fiducial.

In [23]:
dirname = 'double_source_ClY1'

In [24]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
pk2cl.set_cosmology_from_dict()
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=True)
pk2cl.dump_Cl_cache(f'{dirname}/fiducial', overwrite=True)

:20.785980939865112 sec
saving l to double_source_ClY1/fiducial/l.txt
saving lowz,lowz to double_source_ClY1/fiducial/lowz,lowz.txt
saving lowz,cmass1 to double_source_ClY1/fiducial/lowz,cmass1.txt
saving lowz,cmass2 to double_source_ClY1/fiducial/lowz,cmass2.txt
saving cmass1,lowz to double_source_ClY1/fiducial/cmass1,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/fiducial/cmass1,cmass1.txt
saving cmass1,cmass2 to double_source_ClY1/fiducial/cmass1,cmass2.txt
saving cmass2,lowz to double_source_ClY1/fiducial/cmass2,lowz.txt
saving cmass2,cmass1 to double_source_ClY1/fiducial/cmass2,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/fiducial/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY1/fiducial/lowz,s2.txt
saving cmass1,s2 to double_source_ClY1/fiducial/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY1/fiducial/cmass2,s2.txt
saving s2,s2 to double_source_ClY1/fiducial/s2,s2.txt
saving cosmo_dict to double_source_ClY1/fiducial/cosmo_dict.json.
saved galaxy sample lo

In [25]:
cosmo_fiducial = pk2cl.get_cosmo_dict()
dp = 0.01
file_param = od()

In [26]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['omega_b'] = cosmo_fiducial['omega_b']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['omega_b'] = cosmo_fiducial['omega_b']*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/omega_b', overwrite=True)

:12.164162158966064 sec
saving l to double_source_ClY1/omega_b/l.txt
saving lowz,lowz to double_source_ClY1/omega_b/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/omega_b/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/omega_b/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY1/omega_b/lowz,s2.txt
saving cmass1,s2 to double_source_ClY1/omega_b/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY1/omega_b/cmass2,s2.txt
saving s2,s2 to double_source_ClY1/omega_b/s2,s2.txt
saving cosmo_dict to double_source_ClY1/omega_b/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 12.9 s, sys: 98.9 ms, total: 13 s
Wall time: 12.4 s


In [27]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['omega_c'] = cosmo_fiducial['omega_c']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['omega_c'] = cosmo_fiducial['omega_c']*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/omega_c', overwrite=True)

:12.290292024612427 sec
saving l to double_source_ClY1/omega_c/l.txt
saving lowz,lowz to double_source_ClY1/omega_c/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/omega_c/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/omega_c/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY1/omega_c/lowz,s2.txt
saving cmass1,s2 to double_source_ClY1/omega_c/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY1/omega_c/cmass2,s2.txt
saving s2,s2 to double_source_ClY1/omega_c/s2,s2.txt
saving cosmo_dict to double_source_ClY1/omega_c/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 13 s, sys: 110 ms, total: 13.1 s
Wall time: 12.6 s


In [28]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['Omega_de'] = cosmo_fiducial['Omega_de']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['Omega_de'] = cosmo_fiducial['Omega_de']*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/Omega_de', overwrite=True)

:12.15211796760559 sec
saving l to double_source_ClY1/Omega_de/l.txt
saving lowz,lowz to double_source_ClY1/Omega_de/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/Omega_de/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/Omega_de/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY1/Omega_de/lowz,s2.txt
saving cmass1,s2 to double_source_ClY1/Omega_de/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY1/Omega_de/cmass2,s2.txt
saving s2,s2 to double_source_ClY1/Omega_de/s2,s2.txt
saving cosmo_dict to double_source_ClY1/Omega_de/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 12.9 s, sys: 104 ms, total: 13 s
Wall time: 12.4 s


In [29]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['n_s'] = cosmo_fiducial['n_s']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['n_s'] = cosmo_fiducial['n_s']*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/n_s', overwrite=True)

:12.391968965530396 sec
saving l to double_source_ClY1/n_s/l.txt
saving lowz,lowz to double_source_ClY1/n_s/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/n_s/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/n_s/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY1/n_s/lowz,s2.txt
saving cmass1,s2 to double_source_ClY1/n_s/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY1/n_s/cmass2,s2.txt
saving s2,s2 to double_source_ClY1/n_s/s2,s2.txt
saving cosmo_dict to double_source_ClY1/n_s/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 13.1 s, sys: 112 ms, total: 13.2 s
Wall time: 12.7 s


In [30]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['ln10p10As'] = cosmo_fiducial['ln10p10As']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['ln10p10As'] = cosmo_fiducial['ln10p10As']*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/ln10p10As', overwrite=True)

:12.378190755844116 sec
saving l to double_source_ClY1/ln10p10As/l.txt
saving lowz,lowz to double_source_ClY1/ln10p10As/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/ln10p10As/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/ln10p10As/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY1/ln10p10As/lowz,s2.txt
saving cmass1,s2 to double_source_ClY1/ln10p10As/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY1/ln10p10As/cmass2,s2.txt
saving s2,s2 to double_source_ClY1/ln10p10As/s2,s2.txt
saving cosmo_dict to double_source_ClY1/ln10p10As/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 13.1 s, sys: 110 ms, total: 13.2 s
Wall time: 12.7 s


In [31]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78*(1.0+dp), '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['b1lowz'] = 1.78*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/b1lowz', overwrite=True)

[autoreload of hsc3x2pt failed: Traceback (most recent call last):
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<

:12.479421854019165 sec
saving l to double_source_ClY1/b1lowz/l.txt
saving lowz,lowz to double_source_ClY1/b1lowz/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/b1lowz/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/b1lowz/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY1/b1lowz/lowz,s2.txt
saving cmass1,s2 to double_source_ClY1/b1lowz/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY1/b1lowz/cmass2,s2.txt
saving s2,s2 to double_source_ClY1/b1lowz/s2,s2.txt
saving cosmo_dict to double_source_ClY1/b1lowz/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 13.2 s, sys: 110 ms, total: 13.3 s
Wall time: 12.8 s


In [32]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10*(1+dp), '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['b1cmass1'] = 2.10*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/b1cmass1', overwrite=True)

:12.399980783462524 sec
saving l to double_source_ClY1/b1cmass1/l.txt
saving lowz,lowz to double_source_ClY1/b1cmass1/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/b1cmass1/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/b1cmass1/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY1/b1cmass1/lowz,s2.txt
saving cmass1,s2 to double_source_ClY1/b1cmass1/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY1/b1cmass1/cmass2,s2.txt
saving s2,s2 to double_source_ClY1/b1cmass1/s2,s2.txt
saving cosmo_dict to double_source_ClY1/b1cmass1/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 13.1 s, sys: 115 ms, total: 13.2 s
Wall time: 12.7 s


In [33]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28*(1+dp), '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['b1cmass2'] = 2.28*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/b1cmass2', overwrite=True)

using dark_emulator at  /Users/sunao-mac/Documents/python_package/dark_emulator_public/dark_emulator/__init__.py
:12.222837924957275 sec
saving l to double_source_ClY1/b1cmass2/l.txt
saving lowz,lowz to double_source_ClY1/b1cmass2/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/b1cmass2/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/b1cmass2/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY1/b1cmass2/lowz,s2.txt
saving cmass1,s2 to double_source_ClY1/b1cmass2/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY1/b1cmass2/cmass2,s2.txt
saving s2,s2 to double_source_ClY1/b1cmass2/s2,s2.txt
saving cosmo_dict to double_source_ClY1/b1cmass2/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 12.9 s, sys: 102 ms, total: 13 s
Wall time: 12.5 s


In [34]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259*(1.0+dp)])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['alphamaglowz'] = 2.259*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/alphamaglowz', overwrite=True)

[autoreload of hsc3x2pt failed: Traceback (most recent call last):
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<

:12.142527103424072 sec
saving l to double_source_ClY1/alphamaglowz/l.txt
saving lowz,lowz to double_source_ClY1/alphamaglowz/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/alphamaglowz/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/alphamaglowz/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY1/alphamaglowz/lowz,s2.txt
saving cmass1,s2 to double_source_ClY1/alphamaglowz/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY1/alphamaglowz/cmass2,s2.txt
saving s2,s2 to double_source_ClY1/alphamaglowz/s2,s2.txt
saving cosmo_dict to double_source_ClY1/alphamaglowz/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 12.9 s, sys: 99.9 ms, total: 13 s
Wall time: 12.4 s


In [35]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563*(1+dp)])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['alphamagcmass1'] = 3.563*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/alphamagcmass1', overwrite=True)

:12.183211088180542 sec
saving l to double_source_ClY1/alphamagcmass1/l.txt
saving lowz,lowz to double_source_ClY1/alphamagcmass1/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/alphamagcmass1/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/alphamagcmass1/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY1/alphamagcmass1/lowz,s2.txt
saving cmass1,s2 to double_source_ClY1/alphamagcmass1/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY1/alphamagcmass1/cmass2,s2.txt
saving s2,s2 to double_source_ClY1/alphamagcmass1/s2,s2.txt
saving cosmo_dict to double_source_ClY1/alphamagcmass1/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 12.9 s, sys: 104 ms, total: 13 s
Wall time: 12.5 s


In [36]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729*(1+dp)])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['alphamagcmass2'] = 3.729*dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/alphamagcmass2', overwrite=True)

:12.344023942947388 sec
saving l to double_source_ClY1/alphamagcmass2/l.txt
saving lowz,lowz to double_source_ClY1/alphamagcmass2/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/alphamagcmass2/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/alphamagcmass2/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY1/alphamagcmass2/lowz,s2.txt
saving cmass1,s2 to double_source_ClY1/alphamagcmass2/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY1/alphamagcmass2/cmass2,s2.txt
saving s2,s2 to double_source_ClY1/alphamagcmass2/s2,s2.txt
saving cosmo_dict to double_source_ClY1/alphamagcmass2/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 13.1 s, sys: 110 ms, total: 13.2 s
Wall time: 12.6 s


In [37]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', dp, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['dzph1'] = dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/dzph1', overwrite=True)

using dark_emulator at  /Users/sunao-mac/Documents/python_package/dark_emulator_public/dark_emulator/__init__.py
:12.380207061767578 sec
saving l to double_source_ClY1/dzph1/l.txt
saving lowz,lowz to double_source_ClY1/dzph1/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/dzph1/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/dzph1/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY1/dzph1/lowz,s2.txt
saving cmass1,s2 to double_source_ClY1/dzph1/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY1/dzph1/cmass2,s2.txt
saving s2,s2 to double_source_ClY1/dzph1/s2,s2.txt
saving cosmo_dict to double_source_ClY1/dzph1/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 13.1 s, sys: 109 ms, total: 13.2 s
Wall time: 12.7 s


In [38]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', dp, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['dzph2'] = dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/dzph2', overwrite=True)

:12.423188924789429 sec
saving l to double_source_ClY1/dzph2/l.txt
saving lowz,lowz to double_source_ClY1/dzph2/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/dzph2/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/dzph2/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY1/dzph2/lowz,s2.txt
saving cmass1,s2 to double_source_ClY1/dzph2/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY1/dzph2/cmass2,s2.txt
saving s2,s2 to double_source_ClY1/dzph2/s2,s2.txt
saving cosmo_dict to double_source_ClY1/dzph2/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 13.2 s, sys: 111 ms, total: 13.3 s
Wall time: 12.7 s


In [39]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, dp, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['dm1'] = dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/dm1', overwrite=True)

:12.828931093215942 sec
saving l to double_source_ClY1/dm1/l.txt
saving lowz,lowz to double_source_ClY1/dm1/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/dm1/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/dm1/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY1/dm1/lowz,s2.txt
saving cmass1,s2 to double_source_ClY1/dm1/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY1/dm1/cmass2,s2.txt
saving s2,s2 to double_source_ClY1/dm1/s2,s2.txt
saving cosmo_dict to double_source_ClY1/dm1/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 13.5 s, sys: 130 ms, total: 13.6 s
Wall time: 13.1 s


In [40]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, dp, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

file_param['dm2'] = dp

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/dm2', overwrite=True)

using dark_emulator at  /Users/sunao-mac/Documents/python_package/dark_emulator_public/dark_emulator/__init__.py
:12.915732860565186 sec
saving l to double_source_ClY1/dm2/l.txt
saving lowz,lowz to double_source_ClY1/dm2/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/dm2/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/dm2/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY1/dm2/lowz,s2.txt
saving cmass1,s2 to double_source_ClY1/dm2/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY1/dm2/cmass2,s2.txt
saving s2,s2 to double_source_ClY1/dm2/s2,s2.txt
saving cosmo_dict to double_source_ClY1/dm2/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 13.6 s, sys: 129 ms, total: 13.7 s
Wall time: 13.2 s


In [41]:
json.dump(file_param, open(os.path.join(dirname, 'file_param.json'), 'w'), indent=2)

[autoreload of hsc3x2pt failed: Traceback (most recent call last):
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Users/sunao-mac/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<

In [42]:
t1 = time.time()
print(f'{t1-t0} sec')

405.8604860305786 sec
